# Heat transfer 1D

This solve heat transfer.

Assume length of all elements are same.

The heat transfer formula is $c_1d^2T/dx^2+c_2T+c_3$

## Custom data
`n` is number of node.
`L` the length of element
`c` is the coefficients of formula.

In [1]:
import numpy as np

# heat parameters
heat_k = 168
heat_h = 30
heat_p = 12e-3
heat_a = 5e-6
lastF = 20

# parameters
c = [heat_k * heat_a, -heat_h * heat_p, heat_h * heat_p * lastF]
L = 20e-3
n = 5

## solve
You should set your boundary. The default formula is for heat transfer. 

In [2]:
# set up basic stiffness matrix
bigT = np.zeros([n, n])
bigF = np.zeros(n)
for i in range(n - 1):
    bigT[i:i+2, i:i+2] += c[0] / L * np.array([[1, -1], [-1, 1]])
    bigT[i:i+2, i:i+2] -= c[1] * L / 6 * np.array([[2, 1], [1, 2]])
    bigF[i:i+2] += c[2] * L / 2 * np.array([1, 1])

# boundry
bigT[0, :] = 0
bigT[0, 0] = 1
bigF[0] = 100
bigT[-1, -1] += heat_h * heat_a
bigF[-1] += heat_h * heat_a * lastF

# solve
print("bigT")
print(bigT)
print("bigF")
print(bigF)
t = np.linalg.inv(bigT).dot(bigF)
print("Result")
print(t)

# post processing
print("loss")
heat_loss = ((t[1:] + t[:-1]) / 2 - lastF) * heat_h * heat_p * L
heat_loss = [*heat_loss, (t[-1] - lastF) * heat_h * heat_a]
print(heat_loss)
print(np.sum(heat_loss))

bigT
[[ 1.       0.       0.       0.       0.     ]
 [-0.0408   0.0888  -0.0408   0.       0.     ]
 [ 0.      -0.0408   0.0888  -0.0408   0.     ]
 [ 0.       0.      -0.0408   0.0888  -0.0408 ]
 [ 0.       0.       0.      -0.0408   0.04455]]
bigF
[1.00e+02 1.44e-01 1.44e-01 1.44e-01 7.50e-02]
Result
[100.          75.03868587  59.79008101  51.56325515  48.90641549]
loss
[0.48613926912304023, 0.34138356074377463, 0.25687201014282224, 0.21769081427295603, 0.0043359623229528455]
1.306421616605546
